In [309]:
import re
import ollama
from src.data_loading import load_data
from src.parsing import parse_options
from src.rationale_generation import generate_rationale_and_answer
from src.utils import is_rationale_correct
from src.rationalization import rationalize
from src.prompt_generation import create_prompt_examples, create_prompt_set, clean_options, get_correct_answer_text

In [493]:
NUM_PROMPT_EXAMPLES = 5 # number of prompt examples
NUM_EXAMPLES_TO_PROCESS = 10 # number of examples to process

### Load Dataset

In [494]:
ds_train = load_data()

# Step 1: Select the desired range from ds_train
dataset_D = ds_train.select(range(NUM_PROMPT_EXAMPLES, len(ds_train)))

# Step 2: Shuffle the selected dataset
dataset_D_shuffled = dataset_D.shuffle()  # Use any seed you prefer

# Step 3: Select a subset of examples to process
dataset_D_subset = dataset_D_shuffled.select(range(NUM_EXAMPLES_TO_PROCESS))

print(dataset_D_subset)

Dataset({
    features: ['question', 'options', 'rationale', 'correct'],
    num_rows: 10
})


### Create prompt examples

In [495]:
prompt_examples = create_prompt_examples(ds_train, NUM_PROMPT_EXAMPLES)

prompt_examples

[{'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",
  'rationale': 'If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.',
  'answer': 'E'},
 {'question': 'In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?',
  'rationale': 'Line k passes through the origin and has slope 1/5 means that its equation is y=1/5*x.\nThus: (x, 1)=(5, 1) and (5, y) = (5,1) -->x=5 and y=1\nAnswer: C',
  'answer': 'C'},
 {'question': 'For all numbers p and q, the operation @ is defined by p@q = p^2 - pq. If xy ≠ 0, then which of the following can be equal to zero?\nI. x@y\nII. (xy)@y\nIII. x@

### Create Prompt Sets

In [496]:
prompt_set = create_prompt_set(ds_train, NUM_PROMPT_EXAMPLES)
print(prompt_set)

Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?
Answer Explanation: If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.
Answer: 23
###
Question: In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?
Answer Explanation: Line k passes through the origin and has slope 1/5 means that its equation is y=1/5*x.\nThus: (x, 1)=(5, 1) and (5, y) = (5,1) -->x=5 and y=1\nAnswer: C
Answer: 5 and 1
###
Question: For all numbers p and q, the operation @ is defined by p@q = p^2 - pq. If xy ≠ 0, then which of the following can be equal to zero?
I. x@y
II. (xy)@y
III. x@(x + y)
Answer Expl

### Initialize lists to hold correct and incorrect pairs

In [497]:
correct_pairs = []
incorrect_pairs = []
unanswered_paris = []

### Iterate over each example in the subset

In [498]:
def extract_answer_text(rationale):
    """
    Extracts the answer text from the rationale using regex.
    
    Args:
        rationale (str): Generated rationale string.
    
    Returns:
        str or None: Extracted answer text if found, else None.
    """
    # Attempt to extract the answer text after 'Answer:'
    match = re.search(r'Answer:\s*(.+)', rationale, re.IGNORECASE)
    if match:
        return match.group(1)
    return None

In [499]:
for idx, example in enumerate(dataset_D_subset):
    question = example['question']
    # Map 'correct' label to answer text
    raw_options = example['options']
    cleaned_options = clean_options(raw_options)
    correct_label = example['correct'].strip().upper()
    correct_answer_text = get_correct_answer_text(cleaned_options, correct_label)

    print(f"correct_answer_text: {correct_answer_text}")
    print(f"clean options: {cleaned_options}")
    
    if correct_answer_text is None:
        print(f"Skipping example {idx} due to missing correct answer.")
        continue  # Skip this example
    
    # Generate rationale and answer
    generated_rationale = generate_rationale_and_answer(question, prompt_set)
    
    # Extract the answer text from the rationale
    extracted_answer = extract_answer_text(generated_rationale)
    print(f"extracted answer: {extracted_answer}")

    response = ollama.chat(model="llama3.1:8b", messages=[
            {
                'role': 'user',
                'content':
                f"""
                    Your task is to compare two numerical answers and determine if they are the same answer, ignoring differences in units or formatting.\n\n
                    Comparison Rules:\n\n
                    - If the answers are the same, for example, First Answer: '90' and Second Answer: '90 miles' or ( km, %, sec, ml, etc) this is a match and return 'correct' in your response.\n
                    - If the answers are different, consider them NOT a match and return 'incorrect' in your response.\n\n
                    Ignore differences in formatting, such as trailing zeros.\n\n

                    Compare the Following Answers:\n
                    First answer: {extracted_answer}\n
                    Second answer: {correct_answer_text}\n\n
                    Respond with:\n
                    "correct" if the two answers are the same\n
                    "incorrect" if the two answers are not the same\n\n
                    Please respond with only one of the above options, without any explanations.
                """
            },
     ])
    
    decision = response['message']['content'].strip()
    print(decision)
    
    # Categorize based on extracted answer
    if decision.lower() == "correct":
        correct_pairs.append({
            'question': question,
            'rationale': generated_rationale,
            'answer': extracted_answer
        })
        print('Correct:', {
            'question': question,
            'rationale': generated_rationale,
            'answer': extracted_answer
        })
    elif decision.lower() == "incorrect":
        incorrect_pairs.append({
            'question': question,
            'rationale': generated_rationale,
            'answer': extracted_answer if extracted_answer else "No Answer Extracted"
        })
        print("Incorrect:",  {
            'question': question,
            'rationale': generated_rationale,
            'answer': extracted_answer if extracted_answer else "No Answer Extracted"
        })
    else:
        unanswered_paris.append({
            'question': question,
            'rationale': generated_rationale,
            'answer': extracted_answer if extracted_answer else "No Answer Extracted"
        })
        print("Unanswered:",  {
            'question': question,
            'rationale': generated_rationale,
            'answer': extracted_answer if extracted_answer else "No Answer Extracted"
        })
        

    # Print progress every example
    print(f"Processed {idx + 1} questions.\n")

correct_answer_text: 3:2
clean options: ['A) 2:1', 'B) 3:2', 'C) 4:3', 'D) 5:4', 'E) 6:5']
extracted answer: 5:2
incorrect
Incorrect: {'question': 'The marks obtained by Polly and Sandy are in the ratio 3:5 and those obtained by Sandy and Willy are in the ratio of 5:2. The marks obtained by Polly and Willy are in the ratio of...?', 'rationale': "To solve this problem, we need to find the ratio of the marks obtained by Polly and Willy.\n\nLet's first consider the marks obtained by Polly and Sandy, which are in the ratio 3:5. This means that if Polly's marks are 3x, then Sandy's marks are 5x.\n\nNow, let's consider the marks obtained by Sandy and Willy, which are in the ratio 5:2. This means that if Sandy's marks are 5y, then Willy's marks are 2y.\n\nSince Sandy's marks appear in both ratios, we can equate them:\n\n3x = 5y\n\nNow, we need to express y in terms of x:\n\ny = (3/5)x\n\nWe also know that Sandy's marks are 5x. So, substituting the expression for y into the equation for Sandy'

In [491]:
correct_pairs

[{'question': 'A positive number x is multiplied by 8, and this product is then divided by 3. If the positive square root of the result of these two operations equals x, what is the value of x ?',
  'rationale': "Let's solve the problem step-by-step.\n\nStep 1: Understand the given operations\nThe number x is multiplied by 8, resulting in a product of 8x. This product is then divided by 3, giving us (8x)/3.\n\nStep 2: Identify the relationship between the result and x\nWe are told that the positive square root of the result equals x. In other words, √((8x)/3) = x\n\nStep 3: Square both sides to eliminate the square root\nSquaring both sides gives us (8x)/3 = x^2\n\nStep 4: Multiply both sides by 3 to get rid of the fraction\nMultiplying both sides by 3 results in 8x = 3x^2\n\nStep 5: Rearrange the equation to solve for x\nWe can rearrange the equation to get all terms on one side: 3x^2 - 8x = 0\n\nStep 6: Factor out an x from both terms\nFactoring out an x results in x(3x - 8) = 0\n\nS

In [454]:
incorrect_pairs

[{'question': 'A store raised the price of an item by exactly 20 percent. Which of the following could NOT be the resulting price of the item?',
  'rationale': "To solve this problem, let's consider the original price of the item as $100 (any other value would work as well). If the store raises the price by exactly 20%, the new price will be:\n\nOriginal price = $100\nIncrease in price = 20% of $100 = 0.2 x $100 = $20\nNew price = Original price + Increase in price = $100 + $20 = $120\n\nNow, let's examine each option and determine which one could NOT be the resulting price:\n\nOption 1: $125 - This is a possible result if the original price was lower than $100.\n\nOption 2: $110 - This is also a possible result if the original price was higher than $80 but lower than $100.\n\nOption 3: $130 - This could be the new price if the original price was higher than $100, but lower than $120.\n\nNow, let's consider the options that seem less likely:\n\nOption 4: $90 - If the store raises the p

In [455]:
unanswered_paris

[]

In [456]:
total = len(correct_pairs) + len(incorrect_pairs) + len(unanswered_paris)
accuracy = len(correct_pairs) / total * 100
print(f"Total questions processed: {total}")
print(f"Correct answers: {len(correct_pairs)}")
print(f"Incorrect answers: {len(incorrect_pairs)}")
print(f"Unanswered answers: {len(unanswered_paris)}")
print(f"Accuracy: {accuracy:.2f}%")

Total questions processed: 10
Correct answers: 6
Incorrect answers: 4
Unanswered answers: 0
Accuracy: 60.00%


In [26]:
new_correct_pairs = []

# Process the incorrect answers
for pair in incorrect_pairs:
    question = pair['question']
    options = pair['options']
    correct_answer = pair['correct_answer']

    # Generate the rationale with the correct answer as a hint
    generated_rationale = rationalize(question, options, correct_answer, prompt_set)

    # Add the rationalized example to correct_pairs
    new_correct_pairs.append({
        'question': question,
        'options': options,
        'rationale': generated_rationale,
        'answer': correct_answer
    })

    print({
        'question': question,
        'options': options,
        'rationale': generated_rationale,
        'answer': correct_answer
    })

{'question': 'Rs. 825 becomes Rs. 956 in 3 years at a certain rate of simple interest.If the rate of interest is increased by 4% ,What amount will Rs. 825 become in 3 years ?', 'options': {'A': 'Rs. 1020.80', 'B': 'Rs. 1025', 'C': 'Rs. 1055', 'D': 'Data inadequate', 'E': 'None of these'}, 'rationale': "Let's break down the problem step by step.\n\nGiven:\n\n1. Principal (P) = Rs. 825\n2. Amount after 3 years (A) = Rs. 956\n\nWe need to find the rate of interest (R) at which P becomes A in 3 years. We'll use the formula for simple interest:\n\nSimple Interest (SI) = (P × R × T)/100\n\nwhere T is the time period, which is 3 years here.\n\nFirst, let's calculate the SI for the given scenario:\n\nA = P + SI\n956 = 825 + (SI)\nSI = 131\n\nNow, we can use the formula to find the rate of interest (R):\n\nSI = (P × R × T)/100\n131 = (825 × R × 3)/100\n131 = 24.75R\nR = 131/24.75 ≈ 5.29%\n\nThis is the original rate of interest.\n\nNow, let's consider what happens when the rate of interest is i

### Converting our corrected pairs into a structure the Llama3.1 model will understand for fine tuning.

In [141]:
from datasets import Dataset, DatasetDict


def convert_correct_pairs_to_conversations(correct_pairs):
    """
    Converts correct_pairs into the desired conversations format.

    Parameters:
        correct_pairs (list): List of dictionaries containing question, options, correct_answer, and rationale.

    Returns:
        list: A list of conversations formatted for fine-tuning.
    """
    conversations = []
    
    for pair in correct_pairs:
        question = pair['question']
        correct_answer = pair['answer']
        rationale = pair['rationale']
        
        # Construct options text
        # options_text = '\n'.join([f"{key}: {value}" for key, value in options.items()])
        
        # Human prompt
        human_value = (
            f"Question: {question}\n"
        )
        
        # GPT response
        gpt_value = (
            f"{correct_answer}\nRationale: {rationale}"
        )
        
        # Append the conversation as a separate list
        conversation = [
            { "from": "human", "value": human_value },
            { "from": "gpt", "value": gpt_value }
        ]
        
        conversations.append(conversation)
    
    return conversations

# conversations = convert_correct_pairs_to_conversations(correct_pairs)

# # Create a Hugging Face Dataset
# dataset = Dataset.from_dict({"conversations": conversations})

# # Create a DatasetDict with only the 'train' split
# dataset_dict = DatasetDict({
#     "train": dataset
# })

# # Optionally, shuffle the dataset
# dataset_dict["train"] = dataset_dict["train"].shuffle(seed=42)

# # Verify the structure
# print(dataset_dict)
# print(dataset_dict["train"][0])  # Print the first conversation

### Save dataset to JSONL format

In [47]:
import json

def save_dataset_to_jsonl(dataset_dict, file_path='finetuning_data.jsonl'):
    """
    Saves the conversations in dataset_dict to a JSONL file.
    
    Parameters:
        dataset_dict (DatasetDict): The dataset to save.
        file_path (str): The path to the output JSONL file.
        
    Returns:
        None
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        for conversation in dataset_dict["train"]["conversations"]:
            json_line = json.dumps(conversation)
            f.write(json_line + '\n')
    print(f"Dataset successfully saved to {file_path}")

# Usage
save_dataset_to_jsonl(dataset_dict, 'finetuning_data.jsonl')

Dataset successfully saved to finetuning_data.jsonl


### Load dataset from JSONL (for appending new data)

In [ ]:
def load_dataset_from_jsonl(file_path='finetuning_data.jsonl', validation_split=0.1):
    """
    Loads conversations from a JSONL file and splits them into train and validation sets.

    Parameters:
        file_path (str): Path to the JSONL file.
        validation_split (float): Proportion of data to use for validation.

    Returns:
        DatasetDict: A DatasetDict with 'train' and 'validation' splits.
    """
    dataset = Dataset.from_json(file_path)
    dataset = dataset.train_test_split(test_size=validation_split, seed=42)
    dataset_dict = DatasetDict({
        "train": dataset["train"],
        "validation": dataset["test"]
    })
    print(f"Loaded dataset with {len(dataset_dict['train'])} training and {len(dataset_dict['validation'])} validation conversations.")
    return dataset_dict

### Appending New Data

In [500]:
import json

new_conversations = convert_correct_pairs_to_conversations(correct_pairs)

def append_conversations_to_jsonl(conversations, file_path='finetuning_data.jsonl'):
    """
    Appends a list of conversations to an existing JSONL file.

    Parameters:
        conversations (list): List of conversations to append.
        file_path (str): Path to the existing JSONL file.

    Returns:
        None
    """
    with open(file_path, 'a', encoding='utf-8') as f:
        for convo in conversations:
            json_line = json.dumps(convo)
            f.write(json_line + '\n')
    print(f"Successfully appended {len(conversations)} conversations to {file_path}")
    
# Usage
append_conversations_to_jsonl(new_conversations, 'finetuning_data.jsonl')

Successfully appended 4 conversations to finetuning_data.jsonl
